In [5]:
# Выполнил Фомин Степан ПС-31
# Сохранение происходит в 2 таблицы, потому что для Штатов данные отличаются.
# Если какого-то из модулей не хватает, то можно установить их выполнив команду !pip install ...

import numpy as np
import pandas as pd
import os
import git
import sqlalchemy
import pyodbc 

urlGit = 'https://github.com/CSSEGISandData/COVID-19'
dataPath = '../data'
nameDayReportTab = 'CovidDailyReports'
nameDayReportTabUs = 'CovidDailyReportsUs'
dayReport = f"{dataPath}/csse_covid_19_data/csse_covid_19_daily_reports"
nameDayReportPathUs = f"{dataPath}/csse_covid_19_data/csse_covid_19_daily_reports_us"
nameColDayReportTab = ['FIPS', 'Admin2', 'ProvinceState', 'CountryRegion', 'LastUpdate',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'CombinedKey', 'IncidentRate', 'CaseFatalityRatio']
nameColDayReportTabUs = ['ProvinceState', 'CountryRegion', 'LastUpdate', 'Lat', 'Long_', 
       'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'IncidentRate', 'TotalTestResults',
       'PeopleHospitalized', 'CaseFatalityRatio', 'UID', 'ISO3', 'TestingRate', 'HospitalizationRate']

# клонирую репозиторий с git
if not(os.path.exists(dataPath)) or (not os.listdir(dataPath)):
    git.Repo.clone_from(urlGit, dataPath)

# настраиваю подключение и соединяю с БД
nameServer = 'LAPTOP-JOCVFETG'
nameDB = 'CovidData'

from sqlalchemy.engine import URL
strCon = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+nameServer+';DATABASE='+nameDB+';Trusted_Connection=yes;'
urlCon = URL.create("mssql+pyodbc", query={"odbc_connect": strCon})

engine = sqlalchemy.create_engine(urlCon)

# запись данных в 2 таблицы БД, потому что данные для штатов отличаются от остальных
def rememberData(tableName, columnNames, csvFolderPath, engine):
    for filename in os.listdir(csvFolderPath):
        if filename.endswith('.csv'):
            df = pd.read_csv(f"{csvFolderPath}/{filename}", header=None, skiprows=1)
            try:
                df.columns = columnNames
                df.to_sql(
                    name=tableName,
                    con=engine,
                    index=False,
                    ifExists='append'
                )
            except:
                continue

                rememberData(nameDayReportTab, nameColDayReportTab, dayReport, engine)
                rememberData(nameDayReportTabUs, nameColDayReportTabUs, nameDayReportPathUs, engine)

covidDfUs = pd.read_sql('CovidDailyReportsUs', engine)
covidDf = pd.read_sql('CovidDailyReports', engine)

KeyboardInterrupt: ignored

In [4]:
!pip install pyodbc

     |████████████████████████████████| 280 kB 7.2 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287384 sha256=fe4bc6027a78d825bd14956af49b412039838f8f57f32e5d662cb6887a0d7a8d
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc
